# Introduction

In this notebook, one will dive deep into the language models and NLP-based data, and create a model that will detect what is the language of the words in the data. The languages that will be detected will be Portuguese-BR, Spanish-LAT, and English-US. The dataset one will be using will be based on the questions made by the stackoverflow community in the languages aforementioned.

In [1]:
#the data can be found here: https://github.com/alura-cursos/nlp-modelos-linguagem/tree/8d8385f69247064af5b54452b81a5d9a5cbb9846/dataset
import pandas as pd
data_pt=pd.read_csv('https://raw.githubusercontent.com/alura-cursos/nlp-modelos-linguagem/8d8385f69247064af5b54452b81a5d9a5cbb9846/dataset/stackoverflow_portugues.csv')
data_sp=pd.read_csv('https://raw.githubusercontent.com/alura-cursos/nlp-modelos-linguagem/8d8385f69247064af5b54452b81a5d9a5cbb9846/dataset/stackoverflow_espanhol.csv')
data_en=pd.read_csv('https://raw.githubusercontent.com/alura-cursos/nlp-modelos-linguagem/8d8385f69247064af5b54452b81a5d9a5cbb9846/dataset/stackoverflow_ingles.csv')

data_pt.head()

,Id,Título,Questão,Tags,Pontuação,Visualizações
0,2402,Como fazer hash de senhas de forma segura?,"<p>Se eu fizer o <em><a href=""http://pt.wikipe...",<hash><segurança><senhas><criptografia>,350,22367
1,6441,Qual é a diferença entre INNER JOIN e OUTER JOIN?,<p>Qual é a diferença entre <code>INNER JOIN</...,<sql><join>,276,176953
2,579,Por que não devemos usar funções do tipo mysql_*?,<p>Uma dúvida muito comum é por que devemos pa...,<php><mysql>,226,9761
3,2539,As mensagens de erro devem se desculpar?,<p>É comum encontrar uma mensagem de erro que ...,<aplicação-web><gui><console><ux>,214,5075
4,17501,"Qual é a diferença de API, biblioteca e Framew...",<p>Me parecem termos muito próximos e eventual...,<api><framework><terminologia><biblioteca>,193,54191


In [2]:
print(data_pt.Questão[5])

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre><code>//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
</code></pre>

<p>Logo, digamos que o usuário usar a sentença: <code>1); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>

<pre><code>insert into Produtos (coluna) values(1); DROP TABLE Produtos;
</code></pre>

<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code>1</code> e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



Here one detects that the code part of the questions has English words. This will potentially affect the model's detection since the question was made on PT-BR. Also, there are other elements that must be removed, since the punctuation and the html parts can affect the language of the question too.

# Using REGEX


Regular Expressions (regex) are powerful tools for pattern matching and text manipulation. They consist of a sequence of characters representing a search pattern. With just a few symbols, regex can match complex patterns in strings, making them invaluable in programming, data validation, and text processing tasks.

A typical regex includes metacharacters like '.', '*', '+', and '?', allowing for matching any character, repetitions, or optional elements. Character classes like [A-Z] enable matching specific ranges, while '\d' matches any digit. Anchors like '^' and '$' denote the beginning and end of a line.

Regex engines vary, but they generally follow the same principles. When a regex pattern matches a portion of the input text, it returns the position of the match or the matched text itself.

Though powerful, regex can be cryptic and prone to misuse. Properly crafted regex requires practice and understanding, but once mastered, they become an invaluable asset for developers and data analysts alike.

In [3]:
#creating questions only data from the datasets

pt_questions = data_pt.Questão[5]
sp_questions = data_sp.Questão[5]
en_questions = data_en.Questão[5]

import re


Nice! One found the first tag, but still, there are plenty more tags for one to find. There must be a way to find another kind of regular expressions in that specific question.

In [4]:
re.findall(r'<.*>', # use .* to find all texts between <>
           pt_questions)

['<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>',
 '<pre><code>',
 '</code></pre>',
 '<p>Logo, digamos que o usuário usar a sentença: <code>1); DROP TABLE Produtos;</code> para ao campo <code>valor</code> o comando ficaria: </p>',
 '<pre><code>',
 '</code></pre>',
 '<p>Ele vai inserir um novo registro cujo o campo <code>coluna</code> será <code>1</code> e logo em seguida ele vai deletar a tabela Produtos.</p>',
 '<p>Como posso melhorar meu código para prevenir essa situação?</p>']

Using the '.', everything between <> will be returned. One needs to use the '?' to stop this from happening.

In [5]:
re.findall(r'<.*?>', # use .*? to find all texts between <> for once
           pt_questions)

['<p>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<pre>',
 '<code>',
 '</code>',
 '</pre>',
 '<p>',
 '<code>',
 '</code>',
 '<code>',
 '</code>',
 '</p>',
 '<p>',
 '</p>']

In [6]:
#showing the substitution
print(re.sub(r'<.*?>','  T----E----S----T  ',pt_questions))

  T----E----S----T  Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:  T----E----S----T  

  T----E----S----T    T----E----S----T  //----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
  T----E----S----T    T----E----S----T  

  T----E----S----T  Logo, digamos que o usuário usar a sentença:   T----E----S----T  1); DROP TABLE Produtos;  T----E----S----T   para ao campo   T----E----S----T  valor  T----E----S----T   o comando ficaria:   T----E----S----T  

  T----E----S----T    T----E----S----T  insert into Produtos (coluna) values(1); DROP TABLE Produtos;
  T----E----S----T    T----E----S----T  

  T----E----S----T  Ele vai inserir um novo

In [7]:
#showing the real effect
print(re.sub(r'<.*?>','',pt_questions))

Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:

//----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');


Logo, digamos que o usuário usar a sentença: 1); DROP TABLE Produtos; para ao campo valor o comando ficaria: 

insert into Produtos (coluna) values(1); DROP TABLE Produtos;


Ele vai inserir um novo registro cujo o campo coluna será 1 e logo em seguida ele vai deletar a tabela Produtos.

Como posso melhorar meu código para prevenir essa situação?



## The difference between regex and compiled regex

Here one will show the speed of regex code and against a compiled regex. This will show the performance of regex, based on the compilation of the metacharacters.

In [8]:
from timeit import timeit

setup = """import re"""
timeit("""re.search(r'70','3423752637846275623847263746238746283746287570')""",
       setup)


0.6446018999995431

In [9]:
setup = """import re 
regex=re.compile(r'70')"""
timeit("""regex.search('3423752637846275623847263746238746283746287570')""",
       setup)

0.29342129999713507

The timeit executes 10e6 times the code inside it. The compiled regex is more than twice faster than the re.search function. This happens because each time search is executed the regex is called before the search is done. When the regex is compiled it doesn't need to be called on each search. 

# Creating a function to apply regex on the question

In [10]:
def remover(texts, regex):
    if type(texts) == str:
        return regex.sub(' ',texts)
    else:
        return [regex.sub(' ',text) for text in texts]

regex_html = re.compile(r'<.*?>')

tagless_question = remover(pt_questions,regex_html)
print(tagless_question)

 Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo: 

  //----CONSULTA SQL----//
$busca = mysql_query ('insert into Produtos (coluna) values(' . $valor . ')');
  

 Logo, digamos que o usuário usar a sentença:  1); DROP TABLE Produtos;  para ao campo  valor  o comando ficaria:  

  insert into Produtos (coluna) values(1); DROP TABLE Produtos;
  

 Ele vai inserir um novo registro cujo o campo  coluna  será  1  e logo em seguida ele vai deletar a tabela Produtos. 

 Como posso melhorar meu código para prevenir essa situação? 



Now one will remove the code and not just the html tags. The <code></code> is an HTML tag that marks the beginning and the end of the code in question. So if one removes the HTML first no code tag will exist, the code must be removed before the code tag is removed.

In [11]:
def code_remover(texts, regex):
    if type(texts) == str:
        return regex.sub(' ',texts)
    else:
        return [regex.sub(' ',text) for text in texts]
    
#now the regex will find either( a linebreak '\n' or '|'  anything else except the linebreak '.')
regex_code = re.compile(r'<code>(.|(\n))*?</code>')

codeless_question = remover(pt_questions,regex_code)
print(codeless_question)

<p>Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo:</p>

<pre> </pre>

<p>Logo, digamos que o usuário usar a sentença:   para ao campo   o comando ficaria: </p>

<pre> </pre>

<p>Ele vai inserir um novo registro cujo o campo   será   e logo em seguida ele vai deletar a tabela Produtos.</p>

<p>Como posso melhorar meu código para prevenir essa situação?</p>



In [12]:
def html_remover(texts, regex):
    if type(texts) == str:
        return regex.sub(' ',texts)
    else:
        return [regex.sub(' ',text) for text in texts]

regex_html = re.compile(r'<.*?>')

tagless_question = html_remover(codeless_question,regex_html)
print(tagless_question)

 Desenvolvi uma página em PHP para uso interno da empresa que trabalho e apenas pouquíssimas pessoas a utilizam. Através dessa página é possível fazer algumas consultas, inserções, alterações e remoções de dados de uma tabela em um banco de dados MySQL, porém eu acredito que meu código em PHP não está protegido contra injeção de código SQL, por exemplo: 

   

 Logo, digamos que o usuário usar a sentença:   para ao campo   o comando ficaria:  

   

 Ele vai inserir um novo registro cujo o campo   será   e logo em seguida ele vai deletar a tabela Produtos. 

 Como posso melhorar meu código para prevenir essa situação? 



Now the question is clear of any text element that could affect the language detection.

# Transforming all the data.

In [13]:
pt_questions_ = code_remover(data_pt.Questão, regex_code)
pt_questions_clean = html_remover(pt_questions_, regex_html)

sp_questions_ = code_remover(data_sp.Questão, regex_code)
sp_questions_clean = html_remover(sp_questions_, regex_html)

en_questions_ = code_remover(data_en.Questão, regex_code)
en_questions_clean = html_remover(en_questions_, regex_html)

data_pt['clean_questions'] = pt_questions_clean
data_sp['clean_questions'] = sp_questions_clean
data_en['clean_questions'] = en_questions_clean

In [14]:
print(data_pt['clean_questions'] [1])

 Qual é a diferença entre   e  ? Podem me dar alguns exemplos? 



In [15]:
print(data_sp['clean_questions'][5])

 Siempre he visto que en   hay: 

 
 asignaciones   
 comparaciones   y   
 

 Creo entender que   hace algo parecido a comparar el valor de la variable y el   también compara el tipo (como un equals de java).  

  ¿Alguien podría confirmarme este punto y extenderlo? . Soy javero y el no tipado de javascript a veces me encanta y otras lo odio. 

 

 ¿Cuál es la manera correcta en javascript de comparar  ,   y otros valores por defecto?  

   

 ¿    se usa como cadena de texto o como palabra clave? ¿Cual de las siguientes comparaciones es la correcta para un elemento   sin  ? (por ejemplo un label sin contenido) 

   



Now all the dataset is clean. The thing is that Spanish has a unique question mark that must be removed since this will bias the model classification.

In [16]:
print(data_en['clean_questions'] [1])

 I accidentally committed the wrong files to  Git , but I haven't pushed the commit to the server yet. 

  How can I undo those commits from the local repository?   



## Punctuation removal

In [17]:
# \w will remove everything that is not alpha numeric
# \s will allow the spaces and linebreaks to be kept
example = "Dracula: Ah...sarcasm. 'For what profit is it to [a] \nman if he gains the world, and \n\nloses his own soul'?. Matthew 16:26 I believe."

regex_punctuation = re.compile(r'[^\w\s]')
print(remover(example, regex_punctuation))

Dracula  Ah   sarcasm   For what profit is it to  a  
man if he gains the world  and 

loses his own soul    Matthew 16 26 I believe 


## Case removal

In [18]:
example = remover(example, regex_punctuation)

data_pt['clean_questions'] = remover(data_pt['clean_questions'], regex_punctuation)
data_sp['clean_questions'] = remover(data_sp['clean_questions'], regex_punctuation)
data_en['clean_questions'] = remover(data_en['clean_questions'], regex_punctuation)

def case_remover(texts):
    if type(texts) == str:
        return texts.lower()
    else:
        return [text.lower() for text in texts]


data_pt['clean_questions'] = case_remover(data_pt['clean_questions'])
data_sp['clean_questions'] = case_remover(data_sp['clean_questions'])
data_en['clean_questions'] = case_remover(data_en['clean_questions'])

print(case_remover(example))

dracula  ah   sarcasm   for what profit is it to  a  
man if he gains the world  and 

loses his own soul    matthew 16 26 i believe 


## Numbers removal

In [19]:
# \d to remove digits\numbers

example= case_remover(example)
regex_numbers = re.compile(r'\d+')

data_pt['clean_questions'] = remover(data_pt['clean_questions'],regex_numbers)
data_sp['clean_questions'] = remover(data_sp['clean_questions'],regex_numbers)
data_en['clean_questions'] = remover(data_en['clean_questions'],regex_numbers)

print(remover(example, regex_numbers))

dracula  ah   sarcasm   for what profit is it to  a  
man if he gains the world  and 

loses his own soul    matthew     i believe 


## Space and linebreak adjustments

Some questions could have '  ' and \n\n. These can affect the model's "reading ability", since both could be a word to it.

In [20]:

example= remover(example, regex_numbers)
regex_linebreak = re.compile(r'(\n)')
data_pt['clean_questions'] = remover(data_pt['clean_questions'],regex_linebreak)
data_sp['clean_questions'] = remover(data_sp['clean_questions'],regex_linebreak)
data_en['clean_questions'] = remover(data_en['clean_questions'],regex_linebreak)

print(remover(example, regex_linebreak))
example = remover(example, regex_linebreak)

regex_spaces = re.compile(r' {2,}')

data_pt['clean_questions'] = remover(data_pt['clean_questions'],regex_spaces)
data_sp['clean_questions'] = remover(data_sp['clean_questions'],regex_spaces)
data_en['clean_questions'] = remover(data_en['clean_questions'],regex_spaces)

print(re.sub(r' {2,}', ' ',example))

dracula  ah   sarcasm   for what profit is it to  a   man if he gains the world  and   loses his own soul    matthew     i believe 
dracula ah sarcasm for what profit is it to a man if he gains the world and loses his own soul matthew i believe 


In [21]:
#just to make sure everything is ok
print(data_pt['clean_questions'][0])
print(data_sp['clean_questions'][0])
print(data_en['clean_questions'][0])

 se eu fizer o hash de senhas antes de armazená las em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque como força bruta na página de login da aplicação keylogger no cliente e criptoanálise rubberhose qualquer forma de hash não vai impedir esses ataques tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido como dar maior garantia de segurança neste aspecto quais preocupações adicionais evitariam o acesso às senhas existem formas melhores de fazer esse hash 
 las sentencias dinámicas son sentencias sql que se crean como cadenas de texto strings y en las que se insertan concatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas como por ejemplo eso es un ejemplo de una vulnerabi

Now the data is ready to be modelled.

## Fake Char insertion

To detect a language one must calculate the odds of a certain letter being followed by another letter. The model will be based on bigrams made with that language words. Those bigrams will have the odds about letter positions relating to each word. This will help the model understand any new word and the current words it already knows.

In [22]:
#example

from nltk import bigrams
example = re.sub(r' {2,}', ' ',example)
bigrams(example)
print(list(bigrams(example)))


[('d', 'r'), ('r', 'a'), ('a', 'c'), ('c', 'u'), ('u', 'l'), ('l', 'a'), ('a', ' '), (' ', 'a'), ('a', 'h'), ('h', ' '), (' ', 's'), ('s', 'a'), ('a', 'r'), ('r', 'c'), ('c', 'a'), ('a', 's'), ('s', 'm'), ('m', ' '), (' ', 'f'), ('f', 'o'), ('o', 'r'), ('r', ' '), (' ', 'w'), ('w', 'h'), ('h', 'a'), ('a', 't'), ('t', ' '), (' ', 'p'), ('p', 'r'), ('r', 'o'), ('o', 'f'), ('f', 'i'), ('i', 't'), ('t', ' '), (' ', 'i'), ('i', 's'), ('s', ' '), (' ', 'i'), ('i', 't'), ('t', ' '), (' ', 't'), ('t', 'o'), ('o', ' '), (' ', 'a'), ('a', ' '), (' ', 'm'), ('m', 'a'), ('a', 'n'), ('n', ' '), (' ', 'i'), ('i', 'f'), ('f', ' '), (' ', 'h'), ('h', 'e'), ('e', ' '), (' ', 'g'), ('g', 'a'), ('a', 'i'), ('i', 'n'), ('n', 's'), ('s', ' '), (' ', 't'), ('t', 'h'), ('h', 'e'), ('e', ' '), (' ', 'w'), ('w', 'o'), ('o', 'r'), ('r', 'l'), ('l', 'd'), ('d', ' '), (' ', 'a'), ('a', 'n'), ('n', 'd'), ('d', ' '), (' ', 'l'), ('l', 'o'), ('o', 's'), ('s', 'e'), ('e', 's'), ('s', ' '), (' ', 'h'), ('h', 'i'), ('i

Each letter was connected to the next or previous letter. This will allow the model to calculate the odds of that letter being more frequent in certain languages than others. However, this could still bias the model since the first letter of the comments could be the most probable word in any new sentence. The starting word must be defined in any sentence to prevent this bias from happening. This process is called padding with a fake char.

In [23]:
from nltk.lm.preprocessing import pad_both_ends
print(list(bigrams(pad_both_ends(example, n=2))))

[('<s>', 'd'), ('d', 'r'), ('r', 'a'), ('a', 'c'), ('c', 'u'), ('u', 'l'), ('l', 'a'), ('a', ' '), (' ', 'a'), ('a', 'h'), ('h', ' '), (' ', 's'), ('s', 'a'), ('a', 'r'), ('r', 'c'), ('c', 'a'), ('a', 's'), ('s', 'm'), ('m', ' '), (' ', 'f'), ('f', 'o'), ('o', 'r'), ('r', ' '), (' ', 'w'), ('w', 'h'), ('h', 'a'), ('a', 't'), ('t', ' '), (' ', 'p'), ('p', 'r'), ('r', 'o'), ('o', 'f'), ('f', 'i'), ('i', 't'), ('t', ' '), (' ', 'i'), ('i', 's'), ('s', ' '), (' ', 'i'), ('i', 't'), ('t', ' '), (' ', 't'), ('t', 'o'), ('o', ' '), (' ', 'a'), ('a', ' '), (' ', 'm'), ('m', 'a'), ('a', 'n'), ('n', ' '), (' ', 'i'), ('i', 'f'), ('f', ' '), (' ', 'h'), ('h', 'e'), ('e', ' '), (' ', 'g'), ('g', 'a'), ('a', 'i'), ('i', 'n'), ('n', 's'), ('s', ' '), (' ', 't'), ('t', 'h'), ('h', 'e'), ('e', ' '), (' ', 'w'), ('w', 'o'), ('o', 'r'), ('r', 'l'), ('l', 'd'), ('d', ' '), (' ', 'a'), ('a', 'n'), ('n', 'd'), ('d', ' '), (' ', 'l'), ('l', 'o'), ('o', 's'), ('s', 'e'), ('e', 's'), ('s', ' '), (' ', 'h'), (

Now an HTML mark, appears on the starting and ending word of the sentence. With this, the model will know that is the most common letter that starts sentences and ends them.

The process above was made just to show what bigrams are, the real faster way to use them will be made below.

## Language column definition

In [24]:
data_pt['Language']='portuguese'
data_sp['Language']='spanish'
data_en['Language']='english'

# Train and Test data

In [25]:
from sklearn.model_selection import train_test_split

pt_train, pt_test = train_test_split(data_pt.clean_questions, test_size=0.2,
                                     random_state=123)
sp_train, sp_test = train_test_split(data_sp.clean_questions, test_size=0.2,
                                     random_state=123)
en_train, en_test = train_test_split(data_en.clean_questions, test_size=0.2,
                                     random_state=123)


In [26]:
all_pt_questions = ' '.join(pt_train)
all_sp_questions = ' '.join(sp_train)
all_en_questions = ' '.join(en_train)

In [27]:
from nltk.tokenize import WhitespaceTokenizer
all_pt_words = WhitespaceTokenizer().tokenize(all_pt_questions)
all_sp_words = WhitespaceTokenizer().tokenize(all_sp_questions)
all_en_words = WhitespaceTokenizer().tokenize(all_en_questions)

## Applying bigram by pipeline

In [28]:
from nltk.lm.preprocessing import padded_everygram_pipeline

pt_train_bigram, pt_vocab = padded_everygram_pipeline(2, all_pt_words)
sp_train_bigram, sp_vocab = padded_everygram_pipeline(2, all_sp_words)
en_train_bigram, en_vocab = padded_everygram_pipeline(2, all_en_words)
#those variables are generator type

next(next(pt_train_bigram))

('<s>',)

# Using MLE

Maximum Likelihood Estimator (MLE) is a fundamental statistical technique that can be applied in Natural Language Processing (NLP). It aims to estimate parameters for language models by maximizing the likelihood of observing the given data. In NLP, MLE plays a crucial role in tasks like language modeling, part-of-speech tagging, and sentiment analysis. By finding the most probable parameters, MLE allows one to generate more accurate predictions and improve NLP models' performance. This allows the model to comprehend, generate, and process human language effectively.

In [29]:
from nltk.lm import MLE
model_pt = MLE(2)
model_pt.fit(pt_train_bigram, pt_vocab)

model_sp = MLE(2)
model_sp.fit(sp_train_bigram, sp_vocab)

model_en = MLE(2)
model_en.fit(en_train_bigram, en_vocab)

In [30]:
#generating a word based on the model's knowledge
model_en.generate(8)

['s', '</s>', 's', 's', 't', 'h', 'y', '</s>']

In the afore-generated word, what is the most probable char after the letter o?

In [31]:
from nltk.lm import NgramCounter
print(model_en.counts[['o']].items())

dict_items([('w', 509), ('</s>', 1257), ('u', 500), ('g', 65), ('l', 212), ('m', 472), ('n', 1069), ('r', 895), ('e', 87), ('d', 237), ('b', 98), ('f', 375), ('o', 144), ('p', 202), ('t', 303), ('s', 181), ('k', 62), ('v', 103), ('i', 80), ('c', 99), ('x', 21), ('z', 2), ('a', 35), ('j', 19), ('h', 4), ('y', 7)])


The word does not make any sense and the next word probability also don't. Since the aim of this notebook is not text generation, this was only used to show that the model requires more data to generate a better word.

Now one will use the models to see what is the most probable language from the example used before.

In [32]:
example = WhitespaceTokenizer().tokenize(example)
example = [list(pad_both_ends(word, n=2)) for word in example]
example = [list(bigrams(word)) for word in example]

print(example)

[[('<s>', 'd'), ('d', 'r'), ('r', 'a'), ('a', 'c'), ('c', 'u'), ('u', 'l'), ('l', 'a'), ('a', '</s>')], [('<s>', 'a'), ('a', 'h'), ('h', '</s>')], [('<s>', 's'), ('s', 'a'), ('a', 'r'), ('r', 'c'), ('c', 'a'), ('a', 's'), ('s', 'm'), ('m', '</s>')], [('<s>', 'f'), ('f', 'o'), ('o', 'r'), ('r', '</s>')], [('<s>', 'w'), ('w', 'h'), ('h', 'a'), ('a', 't'), ('t', '</s>')], [('<s>', 'p'), ('p', 'r'), ('r', 'o'), ('o', 'f'), ('f', 'i'), ('i', 't'), ('t', '</s>')], [('<s>', 'i'), ('i', 's'), ('s', '</s>')], [('<s>', 'i'), ('i', 't'), ('t', '</s>')], [('<s>', 't'), ('t', 'o'), ('o', '</s>')], [('<s>', 'a'), ('a', '</s>')], [('<s>', 'm'), ('m', 'a'), ('a', 'n'), ('n', '</s>')], [('<s>', 'i'), ('i', 'f'), ('f', '</s>')], [('<s>', 'h'), ('h', 'e'), ('e', '</s>')], [('<s>', 'g'), ('g', 'a'), ('a', 'i'), ('i', 'n'), ('n', 's'), ('s', '</s>')], [('<s>', 't'), ('t', 'h'), ('h', 'e'), ('e', '</s>')], [('<s>', 'w'), ('w', 'o'), ('o', 'r'), ('r', 'l'), ('l', 'd'), ('d', '</s>')], [('<s>', 'a'), ('a', 'n

To test the model one will use perplexity. The lower the value the closer the word is to the model's language.

In [33]:
print(f"Portguese model's perplexicy:\nWord 'for'={model_pt.perplexity(example[3])}\nWord 'what'={model_pt.perplexity(example[4])}\nWord 'profit'={model_pt.perplexity(example[5])}")
print(f"Spanish model's perplexicy:\nWord 'for'={model_sp.perplexity(example[3])}\nWord 'what'={model_sp.perplexity(example[4])}\nWord 'profit'={model_sp.perplexity(example[5])}")
print(f"English model's perplexicy:\nWord 'for'={model_en.perplexity(example[3])}\nWord 'what'={model_en.perplexity(example[4])}\nWord 'profit'={model_en.perplexity(example[5])}")


Portguese model's perplexicy:
Word 'for'=8.868771423894772
Word 'what'=36.397341865405124
Word 'profit'=15.80223293935864
Spanish model's perplexicy:
Word 'for'=10.71834844408005
Word 'what'=26.2454558218365
Word 'profit'=17.594605819820316
English model's perplexicy:
Word 'for'=8.556057461847379
Word 'what'=6.243240369180808
Word 'profit'=9.638932107008605


The words used were: for, what, and profit, and the results are right, those words are in English and the lowest values of perplexity are found in the English model. 

# Some automations and testing the model

This will be the last part when one creates functions to automate some of the processes made above.

In [34]:
def MLE_train(text_list):
    all_questions = ' '.join(text_list)
    all_words = WhitespaceTokenizer().tokenize(all_questions)
    bigrams, vocabulary = padded_everygram_pipeline(2, all_words)
    model = MLE(2)
    model.fit(bigrams, vocabulary)
    return model

In [35]:
model_pt = MLE_train(pt_train)
model_sp = MLE_train(sp_train)
model_en = MLE_train(en_train)

def perplexity(model, text):
    perplexity = 0
    words = WhitespaceTokenizer().tokenize(text)
    words_fakechar = [list(pad_both_ends(word, n=2)) for word in words]
    words_bigrams = [list(bigrams(word)) for word in words_fakechar]
    for word in words_bigrams:
        perplexity += model.perplexity(word)
    return perplexity

### Using the models on the pt_test


In [36]:
print(perplexity(model_pt,pt_test.iloc[0]))
print(perplexity(model_sp,pt_test.iloc[0]))
print(perplexity(model_en,pt_test.iloc[0]))

2010.750019400955
inf
inf


### Using the models on the sp_test

In [37]:
print(perplexity(model_pt,sp_test.iloc[0]))
print(perplexity(model_sp,sp_test.iloc[0]))
print(perplexity(model_en,sp_test.iloc[0]))

inf
711.8401975455689
inf


### Using the models on the en_test

In [38]:
print(perplexity(model_pt,en_test.iloc[0]))
print(perplexity(model_sp,en_test.iloc[0]))
print(perplexity(model_en,en_test.iloc[0]))

389.8361353691403
374.2808748500097
182.14359532129498


Based on the test results one confirms that the models are detecting the language correctly from the first test sample. Sometimes the perplexity could be infinite. This happens because of the probability is = 0, so perplexity is defined by 1/P or 1/0 = inf.

This doesn't mean that the model is very accurate, instead it could mean that the test sample used has words that couldn't be found in the training set. One needs to be alert about that and always use another model to make sure of its conclusions.

# Using Laplace model

This model will allow one to smooth the results increasing the probabilities in a way that the inf values will not appear anymore.

In [43]:
from nltk.lm import Laplace

def laplace_train(text_list):
    all_questions = ' '.join(text_list)
    all_words = WhitespaceTokenizer().tokenize(all_questions)
    bigrams, vocabulary = padded_everygram_pipeline(2, all_words)
    model = Laplace(2)
    model.fit(bigrams, vocabulary)
    return model

model_pt = laplace_train(pt_train)
model_sp = laplace_train(sp_train)
model_en = laplace_train(en_train)

def perplexity(model, text):
    perplexity = 0
    words = WhitespaceTokenizer().tokenize(text)
    words_fakechar = [list(pad_both_ends(word, n=2)) for word in words]
    words_bigrams = [list(bigrams(word)) for word in words_fakechar]
    for word in words_bigrams:
        perplexity += model.perplexity(word)
    return perplexity

print('Testing on portuguese sample test','\n','pt_model ',perplexity(model_pt,pt_test.iloc[0]))
print('sp_model ', perplexity(model_sp,pt_test.iloc[0]))
print('en_model ',perplexity(model_en,pt_test.iloc[0]),'\n')

print('Testing on spanish sample test','\n','pt_model ',perplexity(model_pt,sp_test.iloc[0]))
print('sp_model ',perplexity(model_sp,sp_test.iloc[0]))
print('en_model ',perplexity(model_en,sp_test.iloc[0]),'\n')

print('Testing on english sample test','\n','pt_model ',perplexity(model_pt,en_test.iloc[0]))
print('sp_model ',perplexity(model_sp,en_test.iloc[0]))
print('en_model ',perplexity(model_en,en_test.iloc[0]))

Testing on portuguese sample test 
 pt_model  2011.2813028930555
sp_model  3396.228242347294
en_model  5814.344086715258 

Testing on spanish sample test 
 pt_model  1021.5558410607784
sp_model  718.0246771504738
en_model  1204.7546685979514 

Testing on english sample test 
 pt_model  398.23167300320694
sp_model  378.47899379671964
en_model  182.95105404626915


# Function that knows the text language

In [49]:
def know_language(text_list):
    language  = []
    for text in text_list:
        pt = perplexity(model_pt,text)
        sp = perplexity(model_sp,text)
        en = perplexity(model_en,text)
        if en >= pt <= sp:
            language.append('Portuguese')
        elif pt> en < sp:
            language.append('English')
        else:
            language.append('Spanish')
    return language

pt_results = know_language(pt_test)
print('ACC pt model',(pt_results.count('Portuguese')/len(sp_test))*100)
sp_results = know_language(sp_test)
print('ACC sp model',(sp_results.count('Spanish')/len(sp_test))*100)
en_results = know_language(en_test)
print('ACC en model',(en_results.count('English')/len(sp_test))*100)

ACC pt model 100.0
ACC sp model 96.0
ACC en model 99.0


It seems that the pt model is perfect, but the others are not. This doesn't disqualify the work done, since one had a limited dataset. For NLP and language detection, the datasets used are huge.